In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
raw_data = pd.read_csv('metadata_output.csv')
raw_data = raw_data[raw_data["CDR"] != "None"]
for column in raw_data.columns[32:]:
    raw_data = raw_data[raw_data[column] != "None"]
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3194 entries, 0 to 3358
Data columns (total 32 columns):
Filename             3194 non-null object
SubjectID            3194 non-null object
Visit                3194 non-null object
Group                3194 non-null object
Age                  3194 non-null float64
Sex                  3194 non-null object
MMSE                 3194 non-null object
GDSCALE              3194 non-null object
CDR                  3194 non-null object
NPI-Q                3194 non-null object
FAQ                  3194 non-null object
APOE A1              3194 non-null int64
APOE A2              3194 non-null int64
Study Identifer      3194 non-null int64
Weight               3194 non-null float64
Series Identifer     3194 non-null int64
Acqusisition Type    2751 non-null object
Weighting            3194 non-null object
Pulse Sequence       3194 non-null object
Slice Thickness      3194 non-null float64
TE                   3194 non-null float64
TR         

In [3]:
interest_col = ["MMSE", "GDSCALE", "CDR", "NPI-Q", "APOE A1", "APOE A2", "Weight"]
roi_data = raw_data[interest_col]

In [4]:
roi_data = roi_data.assign(CDR=lambda s: s['CDR'].astype('float'))
roi_data = roi_data.mask(roi_data['CDR'] < 0).dropna()
roi_data = roi_data[roi_data['Weight'] != 0]

In [5]:
features_all = roi_data.mask(roi_data.eq("None")).dropna().astype('float')
features_all_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_all_y = features_all['CDR'].values

features_wo_NPI = roi_data[list(filter(lambda x: x != "NPI-Q", interest_col))]
features_wo_NPI = features_wo_NPI.mask(features_wo_NPI.eq("None")).dropna().astype('float')
features_wo_NPI_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_wo_NPI_y = features_all['CDR'].values

features_wo_GDSCALE = roi_data[list(filter(lambda x: x != "GDSCALE", interest_col))]
features_wo_GDSCALE = features_wo_GDSCALE.mask(features_wo_GDSCALE.eq("None")).dropna().astype('float')
features_wo_GDSCALE_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_wo_GDSCALE_y = features_all['CDR'].values

/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

all_train_x, all_test_x, all_train_y, all_test_y = train_test_split(features_all_x, features_all_y, test_size=0.3)
wo_NPI_train_x, wo_NPI_test_x, wo_NPI_train_y, wo_NPI_test_y = train_test_split(features_wo_NPI_x, features_wo_NPI_y, test_size=0.3)
wo_GDS_train_x, wo_GDS_test_x, wo_GDS_train_y, wo_GDS_test_y = train_test_split(features_wo_GDSCALE_x, features_wo_GDSCALE_y, test_size=0.3)

lab_enc = preprocessing.LabelEncoder()
all_train_y_c = lab_enc.fit_transform(all_train_y)
all_test_y_c = lab_enc.fit_transform(all_test_y)
wo_NPI_train_y_c = lab_enc.fit_transform(wo_NPI_train_y)
wo_NPI_test_y_c = lab_enc.fit_transform(wo_NPI_test_y)
wo_GDS_train_y_c = lab_enc.fit_transform(wo_GDS_train_y)
wo_GDS_test_y_c = lab_enc.fit_transform(wo_GDS_test_y)

# 5-Class problem
## SVM

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc_all = base_estimator=SVC(kernel='rbf').fit(all_train_x, all_train_y_c)
svc_wo_NPI = base_estimator=SVC(kernel='rbf').fit(wo_NPI_train_x, wo_NPI_train_y_c)
svc_wo_GDS = base_estimator=SVC(kernel='rbf').fit(wo_GDS_train_x, wo_GDS_train_y_c)

result_svc_all = svc_all.predict(all_test_x)
result_svc_wo_NPI = svc_wo_NPI.predict(wo_NPI_test_x)
result_svc_wo_GDS = svc_wo_GDS.predict(wo_GDS_test_x)

acc_svc_all = accuracy_score(all_test_y_c, result_svc_all)
acc_svc_wo_NPI = accuracy_score(wo_NPI_test_y_c, result_svc_wo_NPI)
acc_svc_wo_GDS = accuracy_score(wo_GDS_test_y_c, result_svc_wo_GDS)

print("Metric: accuracy")
print("All features: %f" % acc_svc_all)
print("W/o NPI feature: %f" % acc_svc_wo_NPI)
print("W/o GDS feature: %f" % acc_svc_wo_GDS)

/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Metric: accuracy
All features: 0.724324
W/o NPI feature: 0.691892
W/o GDS feature: 0.710811


## Random Forest Classifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_all = RandomForestClassifier(n_estimators=100).fit(all_train_x, all_train_y_c)
rf_wo_NPI = RandomForestClassifier(n_estimators=100).fit(wo_NPI_train_x, wo_NPI_train_y_c)
rf_wo_GDS = RandomForestClassifier(n_estimators=100).fit(wo_GDS_train_x, wo_GDS_train_y_c)

result_rf_all = rf_all.predict(all_test_x)
result_rf_wo_NPI = rf_wo_NPI.predict(wo_NPI_test_x)
result_rf_wo_GDS = rf_wo_GDS.predict(wo_GDS_test_x)

acc_rf_all = accuracy_score(all_test_y_c, result_rf_all)
acc_rf_wo_NPI = accuracy_score(wo_NPI_test_y_c, result_rf_wo_NPI)
acc_rf_wo_GDS = accuracy_score(wo_GDS_test_y_c, result_rf_wo_GDS)

print("Metric: accuracy")
print("All features: %f" % acc_rf_all)
print("W/o NPI feature: %f" % acc_rf_wo_NPI)
print("W/o GDS feature: %f" % acc_rf_wo_GDS)

Metric: accuracy
All features: 0.764865
W/o NPI feature: 0.700000
W/o GDS feature: 0.729730


## Random Forest Regression

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

rfr_all = RandomForestRegressor(n_estimators=100).fit(all_train_x, all_train_y)
rfr_wo_NPI = RandomForestRegressor(n_estimators=100).fit(wo_NPI_train_x, wo_NPI_train_y)
rfr_wo_GDS = RandomForestRegressor(n_estimators=100).fit(wo_GDS_train_x, wo_GDS_train_y)

result_rfr_all = rfr_all.predict(all_test_x)
result_rfr_wo_NPI = rfr_wo_NPI.predict(wo_NPI_test_x)
result_rfr_wo_GDS = rfr_wo_GDS.predict(wo_GDS_test_x)

acc_rfr_all = mean_squared_error(all_test_y_c, result_rfr_all)
acc_rfr_wo_NPI = mean_squared_error(wo_NPI_test_y_c, result_rfr_wo_NPI)
acc_rfr_wo_GDS = mean_squared_error(wo_GDS_test_y_c, result_rfr_wo_GDS)

print("Metric: MSE")
print("All features: %f" % acc_rfr_all)
print("W/o NPI feature: %f" % acc_rfr_wo_NPI)
print("W/o GDS feature: %f\n" % acc_rfr_wo_GDS)


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

lab_enc = preprocessing.LabelEncoder()
result_rfr_all_c = lab_enc.fit_transform([find_nearest([0, 0.5, 1, 2, 3], value) for value in result_rfr_all])
result_rfr_wo_NPI_c = lab_enc.fit_transform([find_nearest([0, 0.5, 1, 2, 3], value) for value in result_rfr_wo_NPI])
result_rfr_wo_GDS_c = lab_enc.fit_transform([find_nearest([0, 0.5, 1, 2, 3], value) for value in result_rfr_wo_GDS])

acc_rfr_all_c = accuracy_score(all_test_y_c, result_rfr_all_c)
acc_rfr_wo_NPI_c = accuracy_score(wo_NPI_test_y_c, result_rfr_wo_NPI_c)
acc_rfr_wo_GDS_c = accuracy_score(wo_GDS_test_y_c, result_rfr_wo_GDS_c)

print("Metric: accuracy")
print("All features: %f" % acc_rfr_all_c)
print("W/o NPI feature: %f" % acc_rfr_wo_NPI_c)
print("W/o GDS feature: %f" % acc_rfr_wo_GDS_c)

Metric: MSE
All features: 0.463977
W/o NPI feature: 0.460952
W/o GDS feature: 0.548220

Metric: accuracy
All features: 0.756757
W/o NPI feature: 0.721622
W/o GDS feature: 0.767568


## Feed Forward Network

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DNN(nn.Module):
    def __init__(self, input_size, num_classes=5):
        super(DNN, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, num_classes),
            # nn.Softmax(),
        )
    
    def forward(self, x):
        y_ = x.float()
        y_ = self.mlp(y_)
        return y_

class MyData(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.len = np.array(X).shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len

In [11]:
from torch.utils.data import DataLoader

batch_size = 16

dnn_all_train = MyData(all_train_x, all_train_y)
dnn_all_train = DataLoader(dataset=dnn_all_train, batch_size=batch_size)
dnn_all_test = MyData(all_test_x, all_test_y)
dnn_all_test = DataLoader(dataset=dnn_all_test, batch_size=batch_size)
dnn_wo_NPI_train = MyData(wo_NPI_train_x, wo_NPI_train_y)
dnn_wo_NPI_train = DataLoader(dataset=dnn_wo_NPI_train, batch_size=batch_size)
dnn_wo_NPI_test = MyData(wo_NPI_test_x, wo_NPI_test_y)
dnn_wo_NPI_test = DataLoader(dataset=dnn_wo_NPI_test, batch_size=batch_size)
dnn_wo_GDS_train = MyData(wo_GDS_train_x, wo_GDS_train_y)
dnn_wo_GDS_train = DataLoader(dataset=dnn_wo_GDS_train, batch_size=batch_size)
dnn_wo_GDS_test = MyData(wo_GDS_test_x, wo_GDS_test_y)
dnn_wo_GDS_test = DataLoader(dataset=dnn_wo_GDS_test, batch_size=batch_size)

In [12]:
model = DNN(all_train_x.shape[1]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

max_epoch = 100
step = 0
for epoch in range(max_epoch):
    for idx, (data, labels) in enumerate(dnn_all_train):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        
        loss = criterion(y_hat, y.long())
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, Loss: {}'.format(epoch, max_epoch, step, loss.item()))
        
        if step % 3000 == 0:
            model.eval()
            acc = 0.
            with torch.no_grad():
                for idx, (data, labels) in enumerate(dnn_all_test):
                    x, y = data.to(DEVICE), labels.to(DEVICE)
                    y_hat = model(x)
                    loss = criterion(y_hat, y.long())
                    _, indices = torch.max(y_hat, dim=-1)
                    acc += torch.sum(indices == y).item()
            print('*'*20, 'Test', '*'*20)
            print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(all_test_y)))
            print('*'*46)
            model.train()
        step += 1

model.eval()
acc = 0.
with torch.no_grad():
    for idx, (data, labels) in enumerate(dnn_all_test):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        loss = criterion(y_hat, y.long())
        _, indices = torch.max(y_hat, dim=-1)
        acc += torch.sum(indices == y).item()
print('*'*20, 'Test', '*'*20)
print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(all_test_y)))
print('*'*46)

Epoch: 0/100, Step: 0, Loss: 3.4647393226623535
******************** Test ********************
Step: 0, Loss: 3.7750091552734375, Accuracy: 0.03513513513513514 %
**********************************************
Epoch: 18/100, Step: 1000, Loss: 0.5738572478294373
Epoch: 37/100, Step: 2000, Loss: 0.4106520116329193
Epoch: 55/100, Step: 3000, Loss: 0.37335407733917236
******************** Test ********************
Step: 3000, Loss: 0.9185576438903809, Accuracy: 0.372972972972973 %
**********************************************
Epoch: 74/100, Step: 4000, Loss: 0.25871700048446655
Epoch: 92/100, Step: 5000, Loss: 0.24780717492103577
******************** Test ********************
Step: 5400, Loss: 0.6830037236213684, Accuracy: 0.3945945945945946 %
**********************************************


In [13]:
model = DNN(wo_NPI_train_x.shape[1]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

max_epoch = 100
step = 0
for epoch in range(max_epoch):
    for idx, (data, labels) in enumerate(dnn_wo_NPI_train):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        
        loss = criterion(y_hat, y.long())
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, Loss: {}'.format(epoch, max_epoch, step, loss.item()))
        
        if step % 3000 == 0:
            model.eval()
            acc = 0.
            with torch.no_grad():
                for idx, (data, labels) in enumerate(dnn_wo_NPI_test):
                    x, y = data.to(DEVICE), labels.to(DEVICE)
                    y_hat = model(x)
                    loss = criterion(y_hat, y.long())
                    _, indices = torch.max(y_hat, dim=-1)
                    acc += torch.sum(indices == y).item()
            print('*'*20, 'Test', '*'*20)
            print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_NPI_test_y)))
            print('*'*46)
            model.train()
        step += 1

model.eval()
acc = 0.
with torch.no_grad():
    for idx, (data, labels) in enumerate(dnn_wo_NPI_test):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        loss = criterion(y_hat, y.long())
        _, indices = torch.max(y_hat, dim=-1)
        acc += torch.sum(indices == y).item()
print('*'*20, 'Test', '*'*20)
print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_NPI_test_y)))
print('*'*46)

Epoch: 0/100, Step: 0, Loss: 1.1583975553512573
******************** Test ********************
Step: 0, Loss: 1.0116751194000244, Accuracy: 0.32972972972972975 %
**********************************************
Epoch: 18/100, Step: 1000, Loss: 0.32060495018959045
Epoch: 37/100, Step: 2000, Loss: 0.5415362119674683
Epoch: 55/100, Step: 3000, Loss: 0.5721131563186646
******************** Test ********************
Step: 3000, Loss: 0.969552755355835, Accuracy: 0.4135135135135135 %
**********************************************
Epoch: 74/100, Step: 4000, Loss: 0.17157284915447235
Epoch: 92/100, Step: 5000, Loss: 0.30896511673927307
******************** Test ********************
Step: 5400, Loss: 1.0236629247665405, Accuracy: 0.3891891891891892 %
**********************************************


In [14]:
model = DNN(wo_GDS_train_x.shape[1]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

max_epoch = 100
step = 0
for epoch in range(max_epoch):
    for idx, (data, labels) in enumerate(dnn_wo_GDS_train):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        
        loss = criterion(y_hat, y.long())
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, Loss: {}'.format(epoch, max_epoch, step, loss.item()))
        
        if step % 3000 == 0:
            model.eval()
            acc = 0.
            with torch.no_grad():
                for idx, (data, labels) in enumerate(dnn_wo_GDS_test):
                    x, y = data.to(DEVICE), labels.to(DEVICE)
                    y_hat = model(x)
                    loss = criterion(y_hat, y.long())
                    _, indices = torch.max(y_hat, dim=-1)
                    acc += torch.sum(indices == y).item()
            print('*'*20, 'Test', '*'*20)
            print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_GDS_test_y)))
            print('*'*46)
            model.train()
        step += 1

model.eval()
acc = 0.
with torch.no_grad():
    for idx, (data, labels) in enumerate(dnn_wo_GDS_test):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        loss = criterion(y_hat, y.long())
        _, indices = torch.max(y_hat, dim=-1)
        acc += torch.sum(indices == y).item()
print('*'*20, 'Test', '*'*20)
print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_GDS_test_y)))
print('*'*46)

Epoch: 0/100, Step: 0, Loss: 1.9047588109970093
******************** Test ********************
Step: 0, Loss: 1.9915257692337036, Accuracy: 0.0 %
**********************************************
Epoch: 18/100, Step: 1000, Loss: 1.0107342004776
Epoch: 37/100, Step: 2000, Loss: 0.30254071950912476
Epoch: 55/100, Step: 3000, Loss: 0.49131038784980774
******************** Test ********************
Step: 3000, Loss: 0.04911867901682854, Accuracy: 0.4135135135135135 %
**********************************************
Epoch: 74/100, Step: 4000, Loss: 0.1822080910205841
Epoch: 92/100, Step: 5000, Loss: 0.3186281621456146
******************** Test ********************
Step: 5400, Loss: 0.02780911698937416, Accuracy: 0.40540540540540543 %
**********************************************


# 3-Class problem

In [5]:
roi_data['CDR'] = roi_data['CDR'].replace(3.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(1.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(0.5, 1.0)

In [16]:
features_all = roi_data.mask(roi_data.eq("None")).dropna().astype('float')
features_all_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_all_y = features_all['CDR'].values

features_wo_NPI = roi_data[list(filter(lambda x: x != "NPI-Q", interest_col))]
features_wo_NPI = features_wo_NPI.mask(features_wo_NPI.eq("None")).dropna().astype('float')
features_wo_NPI_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_wo_NPI_y = features_all['CDR'].values

features_wo_GDSCALE = roi_data[list(filter(lambda x: x != "GDSCALE", interest_col))]
features_wo_GDSCALE = features_wo_GDSCALE.mask(features_wo_GDSCALE.eq("None")).dropna().astype('float')
features_wo_GDSCALE_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_wo_GDSCALE_y = features_all['CDR'].values

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

all_train_x, all_test_x, all_train_y, all_test_y = train_test_split(features_all_x, features_all_y, test_size=0.3)
wo_NPI_train_x, wo_NPI_test_x, wo_NPI_train_y, wo_NPI_test_y = train_test_split(features_wo_NPI_x, features_wo_NPI_y, test_size=0.3)
wo_GDS_train_x, wo_GDS_test_x, wo_GDS_train_y, wo_GDS_test_y = train_test_split(features_wo_GDSCALE_x, features_wo_GDSCALE_y, test_size=0.3)

lab_enc = preprocessing.LabelEncoder()
all_train_y_c = lab_enc.fit_transform(all_train_y)
all_test_y_c = lab_enc.fit_transform(all_test_y)
wo_NPI_train_y_c = lab_enc.fit_transform(wo_NPI_train_y)
wo_NPI_test_y_c = lab_enc.fit_transform(wo_NPI_test_y)
wo_GDS_train_y_c = lab_enc.fit_transform(wo_GDS_train_y)
wo_GDS_test_y_c = lab_enc.fit_transform(wo_GDS_test_y)

/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## SVM

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc_all = base_estimator=SVC(kernel='rbf').fit(all_train_x, all_train_y_c)
svc_wo_NPI = base_estimator=SVC(kernel='rbf').fit(wo_NPI_train_x, wo_NPI_train_y_c)
svc_wo_GDS = base_estimator=SVC(kernel='rbf').fit(wo_GDS_train_x, wo_GDS_train_y_c)

result_svc_all = svc_all.predict(all_test_x)
result_svc_wo_NPI = svc_wo_NPI.predict(wo_NPI_test_x)
result_svc_wo_GDS = svc_wo_GDS.predict(wo_GDS_test_x)

acc_svc_all = accuracy_score(all_test_y_c, result_svc_all)
acc_svc_wo_NPI = accuracy_score(wo_NPI_test_y_c, result_svc_wo_NPI)
acc_svc_wo_GDS = accuracy_score(wo_GDS_test_y_c, result_svc_wo_GDS)

print("Metric: accuracy")
print("All features: %f" % acc_svc_all)
print("W/o NPI feature: %f" % acc_svc_wo_NPI)
print("W/o GDS feature: %f" % acc_svc_wo_GDS)

/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/wooane21/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Metric: accuracy
All features: 0.737838
W/o NPI feature: 0.721622
W/o GDS feature: 0.691892


## Random Forest Classifier

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_all = RandomForestClassifier(n_estimators=100).fit(all_train_x, all_train_y_c)
rf_wo_NPI = RandomForestClassifier(n_estimators=100).fit(wo_NPI_train_x, wo_NPI_train_y_c)
rf_wo_GDS = RandomForestClassifier(n_estimators=100).fit(wo_GDS_train_x, wo_GDS_train_y_c)

result_rf_all = rf_all.predict(all_test_x)
result_rf_wo_NPI = rf_wo_NPI.predict(wo_NPI_test_x)
result_rf_wo_GDS = rf_wo_GDS.predict(wo_GDS_test_x)

acc_rf_all = accuracy_score(all_test_y_c, result_rf_all)
acc_rf_wo_NPI = accuracy_score(wo_NPI_test_y_c, result_rf_wo_NPI)
acc_rf_wo_GDS = accuracy_score(wo_GDS_test_y_c, result_rf_wo_GDS)

print("Metric: accuracy")
print("All features: %f" % acc_rf_all)
print("W/o NPI feature: %f" % acc_rf_wo_NPI)
print("W/o GDS feature: %f" % acc_rf_wo_GDS)

Metric: accuracy
All features: 0.772973
W/o NPI feature: 0.759459
W/o GDS feature: 0.737838


## Random Forest Regression

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

rfr_all = RandomForestRegressor(n_estimators=100).fit(all_train_x, all_train_y)
rfr_wo_NPI = RandomForestRegressor(n_estimators=100).fit(wo_NPI_train_x, wo_NPI_train_y)
rfr_wo_GDS = RandomForestRegressor(n_estimators=100).fit(wo_GDS_train_x, wo_GDS_train_y)

result_rfr_all = rfr_all.predict(all_test_x)
result_rfr_wo_NPI = rfr_wo_NPI.predict(wo_NPI_test_x)
result_rfr_wo_GDS = rfr_wo_GDS.predict(wo_GDS_test_x)

acc_rfr_all = mean_squared_error(all_test_y_c, result_rfr_all)
acc_rfr_wo_NPI = mean_squared_error(wo_NPI_test_y_c, result_rfr_wo_NPI)
acc_rfr_wo_GDS = mean_squared_error(wo_GDS_test_y_c, result_rfr_wo_GDS)

print("Metric: MSE")
print("All features: %f" % acc_rfr_all)
print("W/o NPI feature: %f" % acc_rfr_wo_NPI)
print("W/o GDS feature: %f\n" % acc_rfr_wo_GDS)


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

lab_enc = preprocessing.LabelEncoder()
result_rfr_all_c = lab_enc.fit_transform([find_nearest([0, 1, 2], value) for value in result_rfr_all])
result_rfr_wo_NPI_c = lab_enc.fit_transform([find_nearest([0, 1, 2], value) for value in result_rfr_wo_NPI])
result_rfr_wo_GDS_c = lab_enc.fit_transform([find_nearest([0, 1, 2], value) for value in result_rfr_wo_GDS])

acc_rfr_all_c = accuracy_score(all_test_y_c, result_rfr_all_c)
acc_rfr_wo_NPI_c = accuracy_score(wo_NPI_test_y_c, result_rfr_wo_NPI_c)
acc_rfr_wo_GDS_c = accuracy_score(wo_GDS_test_y_c, result_rfr_wo_GDS_c)

print("Metric: accuracy")
print("All features: %f" % acc_rfr_all_c)
print("W/o NPI feature: %f" % acc_rfr_wo_NPI_c)
print("W/o GDS feature: %f" % acc_rfr_wo_GDS_c)

Metric: MSE
All features: 0.188978
W/o NPI feature: 0.192640
W/o GDS feature: 0.204432

Metric: accuracy
All features: 0.778378
W/o NPI feature: 0.740541
W/o GDS feature: 0.756757


## Feed Forward Network

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DNN(nn.Module):
    def __init__(self, input_size, num_classes=5):
        super(DNN, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, num_classes),
            # nn.Softmax(),
        )
    
    def forward(self, x):
        y_ = x.float()
        y_ = self.mlp(y_)
        return y_

class MyData(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.len = np.array(X).shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len

In [21]:
from torch.utils.data import DataLoader

batch_size = 16

dnn_all_train = MyData(all_train_x, all_train_y)
dnn_all_train = DataLoader(dataset=dnn_all_train, batch_size=batch_size)
dnn_all_test = MyData(all_test_x, all_test_y)
dnn_all_test = DataLoader(dataset=dnn_all_test, batch_size=batch_size)
dnn_wo_NPI_train = MyData(wo_NPI_train_x, wo_NPI_train_y)
dnn_wo_NPI_train = DataLoader(dataset=dnn_wo_NPI_train, batch_size=batch_size)
dnn_wo_NPI_test = MyData(wo_NPI_test_x, wo_NPI_test_y)
dnn_wo_NPI_test = DataLoader(dataset=dnn_wo_NPI_test, batch_size=batch_size)
dnn_wo_GDS_train = MyData(wo_GDS_train_x, wo_GDS_train_y)
dnn_wo_GDS_train = DataLoader(dataset=dnn_wo_GDS_train, batch_size=batch_size)
dnn_wo_GDS_test = MyData(wo_GDS_test_x, wo_GDS_test_y)
dnn_wo_GDS_test = DataLoader(dataset=dnn_wo_GDS_test, batch_size=batch_size)

In [22]:
model = DNN(all_train_x.shape[1]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

max_epoch = 200
step = 0
for epoch in range(max_epoch):
    for idx, (data, labels) in enumerate(dnn_all_train):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        
        loss = criterion(y_hat, y.long())
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, Loss: {}'.format(epoch, max_epoch, step, loss.item()))
        
        if step % 2000 == 0:
            model.eval()
            acc = 0.
            with torch.no_grad():
                for idx, (data, labels) in enumerate(dnn_all_test):
                    x, y = data.to(DEVICE), labels.to(DEVICE)
                    y_hat = model(x)
                    loss = criterion(y_hat, y.long())
                    _, indices = torch.max(y_hat, dim=-1)
                    acc += torch.sum(indices == y).item()
            print('*'*20, 'Test', '*'*20)
            print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(all_test_y)))
            print('*'*46)
            model.train()
        step += 1

model.eval()
acc = 0.
with torch.no_grad():
    for idx, (data, labels) in enumerate(dnn_all_test):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        loss = criterion(y_hat, y.long())
        _, indices = torch.max(y_hat, dim=-1)
        acc += torch.sum(indices == y).item()
print('*'*20, 'Test', '*'*20)
print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(all_test_y)))
print('*'*46)

Epoch: 0/200, Step: 0, Loss: 2.517113208770752
******************** Test ********************
Step: 0, Loss: 3.5288872718811035, Accuracy: 0.31891891891891894 %
**********************************************
Epoch: 18/200, Step: 1000, Loss: 0.6974348425865173
Epoch: 37/200, Step: 2000, Loss: 0.43364399671554565
******************** Test ********************
Step: 2000, Loss: 1.000658631324768, Accuracy: 0.7243243243243244 %
**********************************************
Epoch: 55/200, Step: 3000, Loss: 0.5758708119392395
Epoch: 74/200, Step: 4000, Loss: 0.42220428586006165
******************** Test ********************
Step: 4000, Loss: 0.9092215299606323, Accuracy: 0.7351351351351352 %
**********************************************
Epoch: 92/200, Step: 5000, Loss: 0.4114561676979065
Epoch: 111/200, Step: 6000, Loss: 0.5958482027053833
******************** Test ********************
Step: 6000, Loss: 0.9062435626983643, Accuracy: 0.7351351351351352 %
************************************

In [23]:
model = DNN(wo_NPI_train_x.shape[1]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

max_epoch = 200
step = 0
for epoch in range(max_epoch):
    for idx, (data, labels) in enumerate(dnn_wo_NPI_train):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        
        loss = criterion(y_hat, y.long())
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, Loss: {}'.format(epoch, max_epoch, step, loss.item()))
        
        if step % 2000 == 0:
            model.eval()
            acc = 0.
            with torch.no_grad():
                for idx, (data, labels) in enumerate(dnn_all_test):
                    x, y = data.to(DEVICE), labels.to(DEVICE)
                    y_hat = model(x)
                    loss = criterion(y_hat, y.long())
                    _, indices = torch.max(y_hat, dim=-1)
                    acc += torch.sum(indices == y).item()
            print('*'*20, 'Test', '*'*20)
            print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_NPI_test_y)))
            print('*'*46)
            model.train()
        step += 1

model.eval()
acc = 0.
with torch.no_grad():
    for idx, (data, labels) in enumerate(dnn_wo_NPI_test):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        loss = criterion(y_hat, y.long())
        _, indices = torch.max(y_hat, dim=-1)
        acc += torch.sum(indices == y).item()
print('*'*20, 'Test', '*'*20)
print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_NPI_test_y)))
print('*'*46)

Epoch: 0/200, Step: 0, Loss: 5.73456335067749
******************** Test ********************
Step: 0, Loss: 6.828656196594238, Accuracy: 0.3216216216216216 %
**********************************************
Epoch: 18/200, Step: 1000, Loss: 0.6981197595596313
Epoch: 37/200, Step: 2000, Loss: 0.52468341588974
******************** Test ********************
Step: 2000, Loss: 1.0864758491516113, Accuracy: 0.7351351351351352 %
**********************************************
Epoch: 55/200, Step: 3000, Loss: 0.8135815858840942
Epoch: 74/200, Step: 4000, Loss: 0.7277891635894775
******************** Test ********************
Step: 4000, Loss: 0.8249022364616394, Accuracy: 0.7513513513513513 %
**********************************************
Epoch: 92/200, Step: 5000, Loss: 0.4796340763568878
Epoch: 111/200, Step: 6000, Loss: 0.5599435567855835
******************** Test ********************
Step: 6000, Loss: 0.7297224402427673, Accuracy: 0.7405405405405405 %
******************************************

In [24]:
model = DNN(wo_GDS_train_x.shape[1]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

max_epoch = 200
step = 0
for epoch in range(max_epoch):
    for idx, (data, labels) in enumerate(dnn_wo_GDS_train):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        
        loss = criterion(y_hat, y.long())
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, Loss: {}'.format(epoch, max_epoch, step, loss.item()))
        
        if step % 2000 == 0:
            model.eval()
            acc = 0.
            with torch.no_grad():
                for idx, (data, labels) in enumerate(dnn_all_test):
                    x, y = data.to(DEVICE), labels.to(DEVICE)
                    y_hat = model(x)
                    loss = criterion(y_hat, y.long())
                    _, indices = torch.max(y_hat, dim=-1)
                    acc += torch.sum(indices == y).item()
            print('*'*20, 'Test', '*'*20)
            print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_GDS_test_y)))
            print('*'*46)
            model.train()
        step += 1

model.eval()
acc = 0.
with torch.no_grad():
    for idx, (data, labels) in enumerate(dnn_wo_GDS_test):
        x, y = data.to(DEVICE), labels.to(DEVICE)
        y_hat = model(x)
        loss = criterion(y_hat, y.long())
        _, indices = torch.max(y_hat, dim=-1)
        acc += torch.sum(indices == y).item()
print('*'*20, 'Test', '*'*20)
print('Step: {}, Loss: {}, Accuracy: {} %'.format(step, loss.item(), acc/len(wo_GDS_test_y)))
print('*'*46)

Epoch: 0/200, Step: 0, Loss: 1.7766847610473633
******************** Test ********************
Step: 0, Loss: 1.7486310005187988, Accuracy: 0.021621621621621623 %
**********************************************
Epoch: 18/200, Step: 1000, Loss: 1.053214192390442
Epoch: 37/200, Step: 2000, Loss: 0.7726266980171204
******************** Test ********************
Step: 2000, Loss: 1.2004179954528809, Accuracy: 0.6891891891891891 %
**********************************************
Epoch: 55/200, Step: 3000, Loss: 0.5722869038581848
Epoch: 74/200, Step: 4000, Loss: 0.6979444622993469
******************** Test ********************
Step: 4000, Loss: 1.0030593872070312, Accuracy: 0.7405405405405405 %
**********************************************
Epoch: 92/200, Step: 5000, Loss: 0.4568900465965271
Epoch: 111/200, Step: 6000, Loss: 0.3438292443752289
******************** Test ********************
Step: 6000, Loss: 0.8650738596916199, Accuracy: 0.7162162162162162 %
************************************

# 10-Fold validation with Random Forest

In [18]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

# Data processing
interest_col = ["MMSE", "GDSCALE", "CDR", "NPI-Q", "APOE A1", "APOE A2", "Weight"]
roi_data = raw_data[interest_col]
roi_data = roi_data.assign(CDR=lambda s: s['CDR'].astype('float'))
roi_data = roi_data.mask(roi_data['CDR'] < 0).dropna()
roi_data = roi_data[roi_data['Weight'] != 0]

# 3-class problem
roi_data['CDR'] = roi_data['CDR'].replace(3.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(1.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(0.5, 1.0)

features_all = roi_data.mask(roi_data.eq("None")).dropna().astype('float')
features_all_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_all_y = features_all['CDR'].values

n_fold = 10
kf = KFold(n_splits=n_fold, shuffle=True)
total_acc = 0.
for idx, (train_idx, test_idx) in enumerate(kf.split(features_all_x)):
    X_train, X_test = features_all_x[train_idx], features_all_x[test_idx]
    y_train, y_test = features_all_y[train_idx], features_all_y[test_idx]
    
    rfr = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
    result = [find_nearest([0, 1, 2], value) for value in rfr.predict(X_test)]
    acc = accuracy_score(y_test, result)
    total_acc += acc
    print("Acc %d: %f" % (idx, acc))
    
total_acc /= n_fold
print("Total Acc: %f" % total_acc)

/home/harine/MRI_analysis/myvenv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Acc 0: 0.790323
Acc 1: 0.756098
Acc 2: 0.756098
Acc 3: 0.804878
Acc 4: 0.788618
Acc 5: 0.780488
Acc 6: 0.715447
Acc 7: 0.780488
Acc 8: 0.788618
Acc 9: 0.853659
Total Acc: 0.781471


In [5]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Data processing
interest_col = ["MMSE", "GDSCALE", "CDR", "NPI-Q", "APOE A1", "APOE A2", "Weight"]
roi_data = raw_data[interest_col]
roi_data = roi_data.assign(CDR=lambda s: s['CDR'].astype('float'))
roi_data = roi_data.mask(roi_data['CDR'] < 0).dropna()
roi_data = roi_data[roi_data['Weight'] != 0]

# 3-class problem
roi_data['CDR'] = roi_data['CDR'].replace(3.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(1.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(0.5, 1.0)

features_all = roi_data.mask(roi_data.eq("None")).dropna().astype('float')
features_all_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_all_y = features_all['CDR'].values

n_fold = 10
kf = KFold(n_splits=n_fold, shuffle=True)
total_acc = 0.
for idx, (train_idx, test_idx) in enumerate(kf.split(features_all_x)):
    X_train, X_test = features_all_x[train_idx], features_all_x[test_idx]
    y_train, y_test = features_all_y[train_idx], features_all_y[test_idx]
    
    rfr = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
    result = rfr.predict(X_test)
    acc = accuracy_score(y_test, result)
    total_acc += acc
    print("Acc %d: %f" % (idx, acc))
    
total_acc /= n_fold
print("Total Acc: %f" % total_acc)

/home/harine/MRI_analysis/myvenv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Acc 0: 0.750000
Acc 1: 0.837398
Acc 2: 0.772358
Acc 3: 0.796748
Acc 4: 0.772358
Acc 5: 0.699187
Acc 6: 0.780488
Acc 7: 0.821138
Acc 8: 0.780488
Acc 9: 0.796748
Total Acc: 0.780691


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Data processing
interest_col = ["MMSE", "GDSCALE", "CDR", "NPI-Q", "APOE A1", "APOE A2", "Weight"]
roi_data = raw_data[interest_col]
roi_data = roi_data.assign(CDR=lambda s: s['CDR'].astype('float'))
roi_data = roi_data.mask(roi_data['CDR'] < 0).dropna()
roi_data = roi_data[roi_data['Weight'] != 0]

# 3-class problem
roi_data['CDR'] = roi_data['CDR'].replace(3.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(1.0, 2.0)
roi_data['CDR'] = roi_data['CDR'].replace(0.5, 1.0)

features_all = roi_data.mask(roi_data.eq("None")).dropna().astype('float')
features_all_x = features_all[list(filter(lambda x: x != "CDR", list(features_all.columns)))].values
features_all_y = features_all['CDR'].values

from sklearn.model_selection import train_test_split

all_train_x, all_test_x, all_train_y, all_test_y = train_test_split(features_all_x, features_all_y, test_size=0.3)

clf = LogisticRegression(solver='lbfgs', multi_class='multinomial').fit(all_train_x, all_train_y)
result = clf.predict(all_test_x)
print(accuracy_score(all_test_y, result))
clf.coef_

0.6972972972972973


/home/harine/MRI_analysis/myvenv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/home/harine/MRI_analysis/myvenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([[ 0.32878791, -0.17294972, -0.44251462, -0.90674849, -1.10437898,
        -0.01592924],
       [-0.02147014,  0.08756794,  0.11281184,  0.14310224,  0.18095612,
         0.00638758],
       [-0.30731777,  0.08538179,  0.32970278,  0.76364625,  0.92342286,
         0.00954166]])